In [ ]:
#Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#SPDX-License-Identifier: MIT-0

# Movies Data preparation
### This notebook is used to preprocess and clean the Movies Dataset used for the workshop.

### Install required libraries
The following cell installs required python libraries specified in the 'requirements.txt' file.

In [ ]:
#This cell installs the required libraries specified in the 'requirements.txt' file
!pip install -r requirements.txt --quiet

## Data sourcing

The dataset we're using for that example is "The Movies Dataset".
It contains metadata on over 45,000 movies. 26 million ratings from over 270,000 users and can be downloaded here:
https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset

The dataset has a "CC0: Public Domain" license
https://creativecommons.org/publicdomain/zero/1.0/


## Download and extract the dataset

IMPORTANT: Note that you will need to authenticate in kaggle to download the file manually. 
https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset


Then, copy the downloaded zip into the "dataset" folder. 

In [ ]:
import zipfile
import pandas as pd

This cell sets options for displaying data frames in Pandas to show all rows and columns.

In [ ]:
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

This code sets the path for the dataset folder and the ZIP file.

In [ ]:
dataset_path = "../dataset"

#check the name of the downloaded zip file i.e. archive.zip and update below line to reflect the correct zip file name
filename = "archive.zip"
zip_file = f'{dataset_path}/{filename}'

This code extracts the contents of the ZIP file into the dataset folder.

In [ ]:
with zipfile.ZipFile(zip_file) as zf:
   zf.extractall(dataset_path)

This code reads the CSV files from the dataset folder into separate Pandas dataframes.

In [ ]:
#open files as dataframes
credits_df = pd.read_csv('../dataset/credits.csv')
keywords_df = pd.read_csv('../dataset/keywords.csv')
links_small_df = pd.read_csv('../dataset/links_small.csv')
links_df = pd.read_csv('../dataset/links.csv')
movies_metadata_df = pd.read_csv('../dataset/movies_metadata.csv')
ratings_small_df = pd.read_csv('../dataset/ratings_small.csv')
ratings_df = pd.read_csv('../dataset/ratings.csv')

This code removes rows with missing values in the 'popularity' column and converts the data type of 'popularity' to numeric.

In [ ]:
#Popularity Columns (10) have mixed types, removing rows with popularity 
print(movies_metadata_df.shape)

movies_metadata_df["popularity"] = pd.to_numeric(movies_metadata_df["popularity"], errors="coerce")
movies_metadata_df = movies_metadata_df.dropna(subset=["popularity"])

print(movies_metadata_df.shape)

This code displays the first two rows of the 'movies_metadata_df' dataframe.

In [ ]:
movies_metadata_df.head(2)

This code adds a 'keywords' column to the 'movies_metadata_df' dataframe by extracting keywords from the 'keywords_df' dataframe.

In [ ]:
import ast
def add_keywords(id):
    result = []
    try:
        keywords = keywords_df.loc[keywords_df['id'] == int(id), "keywords"].iloc[0]
        list_of_dicts = ast.literal_eval(keywords)
        for _dict in list_of_dicts:
            result.append(_dict['name'])
        return ','.join(result)
    except Exception as e:
        print(f"Exception:{e}")
        return ""

#some rows have data quality issues, expect exceptions to be raised.
movies_metadata_df['keywords'] = movies_metadata_df['id'].apply(lambda x: add_keywords(x))

This code adds a 'director' column to the 'movies_metadata_df' dataframe by extracting director information from the 'credits_df' dataframe.

In [ ]:
def add_director(id):
    director = ''
    try:
        crew = credits_df.loc[credits_df['id'] == int(id), "crew"].iloc[0]
        list_of_dicts = ast.literal_eval(crew)
        for _dict in list_of_dicts:
            if _dict['job'] == 'Director':
                director = _dict['name']
                break
        return director
    except Exception as e:
        print(e)
        return director
    
#some rows have data quality issues, expect exceptions to be raised.
movies_metadata_df['director'] = movies_metadata_df['id'].apply(lambda x: add_director(x))

This code adds an 'actors' column to the 'movies_metadata_df' dataframe by extracting the top 3 actors from the 'credits_df' dataframe.

In [ ]:
def add_actors(id):
    actors = []
    try:
        cast = credits_df.loc[credits_df['id'] == int(id), "cast"].iloc[0]
        list_of_dicts = ast.literal_eval(cast)
        counter = 0
        for _dict in list_of_dicts:
            if _dict['order'] in [0,1,2]:
                actors.append(_dict['name'])
                counter += 1
                if counter == 3:
                    break
        return ",".join(actors)
    except Exception as e:
        print(e)
        return ""

#some rows have data quality issues, expect exceptions to be raised.
movies_metadata_df['actors'] = movies_metadata_df['id'].apply(lambda x: add_actors(x))

This code rounds the 'popularity' column values to one decimal place and converts the data type to float.

In [ ]:
import math

#round value to first decimal
def update_popularity(num):
  try:
    return round(num, 1)
  except Exception as e:
    return 0

movies_metadata_df['popularity'] = movies_metadata_df['popularity'].apply(lambda x: update_popularity(x))

#casting as integer
movies_metadata_df["popularity"] = movies_metadata_df["popularity"].astype(float)

This code adds a 'popularity_bins' column to the 'movies_metadata_df' dataframe by binning the 'popularity' values into five categories.

In [ ]:
movies_metadata_df["popularity_bins"] = pd.qcut(movies_metadata_df['popularity'], 5, labels=['Very Low', 'Low', 'Average', 'High', 'Very High'], duplicates='drop')

This code adds a 'vote_average_bins' column to the 'movies_metadata_df' dataframe by binning the 'vote_average' values into five categories.

In [ ]:
movies_metadata_df["vote_average_bins"] = pd.qcut(movies_metadata_df['vote_average'], 5, labels=['Very Low', 'Low', 'Average', 'High', 'Very High'], duplicates='drop')

This code updates the 'genres' column in the 'movies_metadata_df' dataframe by extracting genre names from the JSON data.

In [ ]:
def update_genre(json_genre):
    genres = []
    try:
        list_of_dicts = ast.literal_eval(json_genre)
        for _dict in list_of_dicts:
            genres.append(_dict['name'])
        return ",".join(genres)
    except Exception as e:
        print(e)
        return ""

movies_metadata_df['genres'] = movies_metadata_df['genres'].apply(lambda x: update_genre(x))

This code adds a 'year' column to the 'movies_metadata_df' dataframe by extracting the year from the 'release_date' column.

In [ ]:
def create_year(x):
    try:
        #expected format 1995-10-30
        return x.split('-')[0]
    except Exception as e:
        return ""

movies_metadata_df['year'] = movies_metadata_df['release_date'].apply(lambda x: create_year(x))

This code displays the null count for each column in the 'movies_metadata_df' dataframe.

In [ ]:
for col in movies_metadata_df.columns:
  null_count = movies_metadata_df[col].isnull().sum()
  print(f"Null count in {col}: {null_count}")

This code displays the column names of the 'movies_metadata_df' dataframe.

In [ ]:
movies_metadata_df.columns

This code renames the 'id' and 'overview' columns, selects relevant columns, and creates two new dataframes: 'to_export_df_full' (containing all rows) and 'to_export_df_small' (containing the top 200 rows sorted by popularity).

In [ ]:
#renaming the id column to tmdb_id
movies_metadata_df.rename(columns={'id': 'tmdb_id'}, inplace=True)

#renaming the overview column to description
movies_metadata_df.rename(columns={'overview': 'description'}, inplace=True)

cols_selection = ['tmdb_id', 'original_language', 'original_title', 'description', 'genres', 'year', 'keywords', 'director', 'actors', 'popularity', 'popularity_bins',
                  'vote_average', 'vote_average_bins']

#full 45K dataset
to_export_df_full = movies_metadata_df[cols_selection]

This code displays the first 10 rows of the 'to_export_df_small' dataframe.

In [ ]:
#small version for workshop's purpose. we're selection the top 200 most popular.
to_export_df_small = to_export_df_full.sort_values(by=['popularity'], ascending=[False])[:200]

This code displays the first 10 rows of the 'to_export_df_small' dataframe.

In [ ]:
to_export_df_small.head(10)

This code exports the concise('to_export_df_small') and full('to_export_df_full') datasets as CSV files in the dataset folder.
For the remaining part of this workshop, we will be working with the concise data set.

In [ ]:
#export
to_export_df_small.to_csv('../dataset/movies_metadata_small.csv', index=False)
to_export_df_full.to_csv('../dataset/movies_metadata_45K.csv', index=False)